In [ ]:
import pandas as pd, numpy as np, xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON
import scipy.io

In [ ]:
import plotly
plotly.offline.init_notebook_mode()
plotly_config = {'scrollZoom': True, 'displaylogo': False, 'toImageButtonOptions': {
    'format': 'svg', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': None,
    'width': None,
    'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
  },
  'modeBarButtonsToAdd': 
    ['drawline',
    'drawopenpath',
    'drawclosedpath',
    'drawcircle',
    'drawrect',
    'eraseshape'
    ]
  
  }

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
config_path = Path(params["config_path"])
config = config_adapter.load(config_path)
RenderJSON(config)

In [ ]:
fs, data = scipy.io.wavfile.read(params["audio_path"])
song = xr.Dataset()
song["data"] = xr.DataArray(data, dims="t")
song["t"] = np.arange(data.size)/fs
song["t"].attrs["fs"] = fs
song

In [ ]:
volume_params = config["volume"]
win_size = int(np.round(volume_params["window_duration"]*fs))
stride = int(np.round(fs/volume_params["approx_out_fs"]))
if volume_params["window_type"] == "hanning":
    window = xr.DataArray(np.hanning(win_size), dims="window")
else:
    raise Exception(f'Unhandled windowtype {volume_params["window_type"]}')
tmp = song["data"].rolling(t=win_size, center=True).construct("window", stride = stride).dropna(dim="t", how="any")
volume = xr.Dataset()
volume["volume"] =  np.log10(np.abs(tmp * window).mean("window"))
volume_fs = fs/stride
volume["t"].attrs["fs"] = volume_fs
volume

In [ ]:
win_size = config["spectrogram"]["nfft"]
stride = config["spectrogram"]["hop"]
spectro_window = song["data"].rolling(t=win_size, min_periods=win_size, center=True).construct("window_t", stride=stride)
if config["spectrogram"]["window_type"] =="hanning":
    spectro_window = spectro_window * xr.DataArray(np.hanning(spectro_window.sizes["window_t"]), dims="window_t")
else: raise Exception(f'Unknown window type {config["spectrogram"]["window_type"]}')
fft = xr.apply_ufunc(np.fft.rfft, spectro_window, input_core_dims=[["window_t"]], output_core_dims=[["f"]])
fft["f"] = np.fft.rfftfreq(spectro_window.sizes["window_t"], 1/fs)
fft = fft.sel(f=slice(config["spectrogram"]["f_bounds"][0], config["spectrogram"]["f_bounds"][1]))
psd = np.abs(fft)**2
display_psd = np.log10(psd)
display_psd

In [ ]:
if "annotation_path" in params:
    annotations = pd.read_csv(params["annotation_path"]).rename(columns={"name": "label", "start_seconds": "start", "stop_seconds": "end"}).sort_values("start")
else:
    annotations = pd.DataFrame([], columns=["label", "start", "end"])
annotations

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3], shared_xaxes=True)
max = display_psd.max().item()
fig.add_trace(go.Heatmap(
    z=display_psd.transpose("f", "t").values, 
    x=display_psd["t"].values, 
    y=display_psd["f"].values,
    hovertemplate ="""
          <b>t: %{x}s</b>
          f: %{y}Hz
          val: %{z} (log10(fft**2))
        """.replace('\n', '<br>'),
    zmin=max/2, zmax=max, name="spectrogram", colorbar=dict(len=0.7), colorbar_y=0.7),
    row=1, col=1)
fig.add_trace(go.Scatter(
    x=volume["t"].values, 
    y=volume["volume"].values,
    hovertemplate ="""
          <b>t: %{x}s</b>
          val: %{y} (log10(mean(abs(window))))
        """.replace('\n', '<br>'),
    name="volume",
    showlegend=False,),
    row=2, col=1)
if "syb_annotations" in params:
    annot_table = annotations.assign(t=(annotations["start"]+annotations["end"])/2)[["t"]+params["syb_annotations"]]
    hover_template = ''.join([f'<br>\t{p}: '+ '%{customdata[' + str(i) + ']}' for i, p in enumerate(params["syb_annotations"])])
    fig.add_trace(go.Scatter(
        x=annot_table["t"],
        y=[display_psd["f"].mean().item()] * len(annot_table.index) ,
        customdata= annot_table[params["syb_annotations"]],
        mode='lines',
        opacity=0,
        hovertemplate =hover_template,
        showlegend=False,
        name="syb_info"
), row=1, col=1)
    
if not "other_bounds" in params:
    params["other_bounds"] = []
for row in annotations.to_dict(orient="index").values():
    fig.add_vrect(x0=row["start"], x1=row["end"], 
                label = dict(
                    text=row["label"],
                    textposition="top center",
                    font=dict(size=20, family="Times New Roman", color="white"),
                ),
                line=dict(color="MediumPurple"))
    for d in params["other_bounds"]:
        fig.add_vrect(x0=row[d["start"]], x1=row[d["end"]], 
                line=dict(color="yellow", dash="dot"))
    
fig.update_traces(xaxis='x')
fig.update_shapes(selector=dict(type="rect"), xref="x")

fig.update_layout(hovermode='x unified', hoversubplots="axis", xaxis_showticklabels=True)
fig.show(config = plotly_config)